In [2]:
import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import json
import time
import pickle
import os
import skimage as sk
from skimage import io
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
from overall import *
import smtplib
import json
import imghdr
from email.message import EmailMessage
from skimage.transform import resize

with open('secret.json') as json_file:
    env_variables = json.load(json_file)
    os.environ['EMAIL_ADD'] = env_variables['EMAIL_ADD']
    os.environ['EMAIL_PASS'] = env_variables['EMAIL_PASS']
    os.environ['AZURE'] = env_variables['AZURE']

In [3]:
azure_key = os.environ.get('AZURE')
EMAIL_ADD = os.environ.get('EMAIL_ADD')
EMAIL_PASS = os.environ.get('EMAIL_PASS')
blob_service_client = BlobServiceClient.from_connection_string(azure_key)

In [4]:
def im_resize(fname):
    img = io.imread(fname)
    img = resize(img, (800, 600, 3))
    io.imsave(fname, img)

In [5]:
containers = blob_service_client.list_containers() 
cs = {}
for c in containers:
    cs[c.name] = c.last_modified
container_name = sorted(cs, key=cs.get, reverse=True)[0]

In [6]:
def send_email(file_name):
    msg = EmailMessage()
    msg['Subject'] = 'Rectify: Item wrongly recycled'
    msg['From'] = EMAIL_ADD
    msg['To'] = 'rectify.hackathon@gmail.com'
    msg.set_content('Oops, you have incorrectly placed an apple into recycling ;)')

    with open(file_name, 'rb') as f:
        file_data = f.read()
        file_type = imghdr.what(f.name)
        file_name = f.name
    msg.add_attachment(file_data, maintype='image', subtype=file_type, filename=file_name)

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:

        smtp.login(EMAIL_ADD, EMAIL_PASS)

        # Args: sender, reciever, message
        smtp.send_message(msg)

In [7]:
def check_recycle(objects, fname):
    recycle_dict = ['water_bottle', 'mixing_bowl']
    flag = False
    for obj in objects:
        if obj not in recycle_dict:
            flag = True
    if flag:
        print("Non-recyclable object detected! Rectifying...\n\n")
        send_email(fname)

In [8]:
image_history = []
obj_list_history = []
SSIM_THRESH = 0.5

while True:
    time.sleep(1)
    # new container client
    container_client = blob_service_client.get_container_client(container_name)
    container_client.list_blobs()
    blobs = []
    for i in container_client.list_blobs():
        blobs.append(i)
    #print("Num blobs: {}".format(len(blobs)))
    
    for blob in container_client.list_blobs():
        name = blob['name']
        if name not in image_history:
            blob_client = blob_service_client.get_blob_client(container=container_name, blob=name)
            # save current image
            this_name = 'real_time/{}.jpg'.format(int(time.time()))
            with open(this_name, 'wb') as f:
                f.write(blob_client.download_blob().readall())
                
            time.sleep(1)
            if len(os.listdir('real_time'))!=0:
                last_img = sorted(os.listdir('real_time'))[-2]
                if last_img != '.DS_Store':
                    last_img = io.imread('real_time/{}'.format(last_img))
                    current_img_name = sorted(os.listdir('real_time'))[-1]
                    current_img = io.imread('real_time/{}'.format(current_img_name))
                    sim_idx = ssim(last_img, current_img, multichannel=True)
                    print(sim_idx)
                    if sim_idx <= 0.8:
                        print("New event found!!!!")
                        fname = 'real_time/{}'.format(current_img_name)
                        obj_list = get_object_list(fname)
                        new_objs = [i for i in obj_list if i not in obj_list_history]
                        obj_list_history = obj_list_history + new_objs
                        check_recycle(new_objs, fname)
                        print(obj_list_history)
            image_history.append(name)

0.7815365169736449
New event found!!!!


tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_1:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_2:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_3:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_4:0' shape=(9, 4) dtype=float32> anchors
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Non-recyclable object detected! Rectifying...


['orange']
0.7849440773938005
New event found!!!!
tracking <tf.Variable 'Variable_5:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_6:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_7:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_8:0' shape=(9, 4) dtype=float32> a

KeyboardInterrupt: 